<center><img src="Imágenes/Logo Siniestros Rezise.png" alt="logo"></center>


<h5 style="color:blue">
Presentado por el Alumno: Javier Edgar Esteban 
Ghithub: JavierEdgarEsteban77 
Carrera: DataScience Cohorte 05PT 02 Año 2.023</h5>

Realizaremos un análisis exploratorio pormenorizado de los datos que nos provee el Sistema Nacional de información Criminal dependiente del Ministerio de Seguridad de la Nación Argentina entre los periodos que comprenden desde el año 2.016 al 2.022

### Instalo paquetes e importo librerías necesarias para comenzar con mi trabajo de analytics.

In [1]:
# Instalo paquetes
%pip install geopandas
%pip install geojson
%pip install folium 
%pip install utils
%pip install bs4
%pip install openpyxl
%pip install pyrae


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

In [2]:
# Importo librerías.
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geojson
import folium
import re
import requests
import utils
import urllib3
import datetime
import string
from shapely.wkt import loads
from shapely.geometry import Point
from folium.plugins import HeatMap
from bs4 import BeautifulSoup
from tqdm import tqdm
from typing import List
from pyrae import dle


### Importo los datos para comenzar a trabajar con ellos. Trabajaremos con homicidios y lesiones.

#### Lo que haré es ver el contenido del archivo xlsx y si tienen hojas, y en tal caso, visualizar dicha información.

In [3]:
# Rutas de archivos
ruta_homicidios = r'C:\Users\Esteban García\OneDrive\Escritorio\LABs\PI02\Datasets Originales\homicidios.xlsx'
ruta_lesiones = r'C:\Users\Esteban García\OneDrive\Escritorio\LABs\PI02\Datasets Originales\lesiones.xlsx'

# Archivos provistos por CABA
homicidios = pd.read_excel(ruta_homicidios)
lesiones = pd.read_excel(ruta_lesiones)

# Leo y exploro hojas y columnas de homicidios.xlsx
homicidios_sheets = pd.ExcelFile(ruta_homicidios)

print("Explorando homicidios.xlsx:")
# Uso tqdm para agregar barras de progreso
for sheet_name in tqdm(homicidios_sheets.sheet_names, desc="Hoja"):
    sheet_data = pd.read_excel(homicidios_sheets, sheet_name)
    print(f"\nArchivo: homicidios - Hoja: {sheet_name}\nColumnas:")
    print(sheet_data.columns)

# Leo las hojas del archivo homicidios.xlsx
homicidios_hechos = pd.read_excel(ruta_homicidios, sheet_name='HECHOS')
homicidios_victimas = pd.read_excel(ruta_homicidios, sheet_name='VICTIMAS')

# Visualizo contenido de la hoja HECHOS
print("\nContenido de la hoja HECHOS en homicidios.xlsx:")
print(homicidios_hechos.head())

# Visualizo contenido de la hoja VICTIMAS
print("\nContenido de la hoja VICTIMAS en homicidios.xlsx:")
print(homicidios_victimas.head())

# Leo y exploro hojas y columnas de lesiones.xlsx
lesiones_sheets = pd.ExcelFile(ruta_lesiones)

print("\nExplorando lesiones.xlsx:")
# Uso tqdm para agregar barras de progreso
for sheet_name in tqdm(lesiones_sheets.sheet_names, desc="Hoja"):
    sheet_data = pd.read_excel(lesiones_sheets, sheet_name)
    print(f"\nArchivo: lesiones - Hoja: {sheet_name}\nColumnas:")
    print(sheet_data.columns)

# Leo las hojas del archivo lesiones.xlsx
lesiones_hechos = pd.read_excel(ruta_lesiones, sheet_name='HECHOS')
lesiones_victimas = pd.read_excel(ruta_lesiones, sheet_name='VICTIMAS')

# Visualizo contenido de la hoja HECHOS
print("\nContenido de la hoja HECHOS en lesiones.xlsx:")
print(lesiones_hechos.head())

# Visualizo contenido de la hoja VICTIMAS
print("\nContenido de la hoja VICTIMAS en lesiones.xlsx:")
print(lesiones_victimas.head())


Explorando homicidios.xlsx:


Hoja:  20%|██        | 1/5 [00:02<00:08,  2.20s/it]


Archivo: homicidios - Hoja: HECHOS
Columnas:
Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

Archivo: homicidios - Hoja: DICCIONARIO_HECHOS
Columnas:
Index(['variables y definiciones', 'descripción', 'Unnamed: 2'], dtype='object')

Archivo: homicidios - Hoja: VICTIMAS
Columnas:
Index(['ID_hecho', 'FECHA', 'AAAA', 'MM', 'DD', 'ROL', 'VICTIMA', 'SEXO',
       'EDAD', 'FECHA_FALLECIMIENTO'],
      dtype='object')


Hoja: 100%|██████████| 5/5 [00:02<00:00,  2.07it/s]



Archivo: homicidios - Hoja: DICCIONARIO_VICTIMAS
Columnas:
Index(['variables y definiciones', 'descripción', 'Unnamed: 2'], dtype='object')

Archivo: homicidios - Hoja: clas
Columnas:
Index(['Original_victima', 'LESIONES', 'FATALES INFORME ANUAL', 'Unnamed: 3',
       'Original_acusado', 'LESIONES.1', 'FATALES INFORME ANUAL.1'],
      dtype='object')

Contenido de la hoja HECHOS en homicidios.xlsx:
          ID  N_VICTIMAS      FECHA  AAAA  MM  DD      HORA HH  \
0  2016-0001           1 2016-01-01  2016   1   1  04:00:00  4   
1  2016-0002           1 2016-01-02  2016   1   2  01:15:00  1   
2  2016-0003           1 2016-01-03  2016   1   3  07:00:00  7   
3  2016-0004           1 2016-01-10  2016   1  10  00:00:00  0   
4  2016-0005           1 2016-01-21  2016   1  21  05:20:00  5   

                             LUGAR_DEL_HECHO TIPO_DE_CALLE  ...  Altura  \
0  AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ       AVENIDA  ...     NaN   
1          AV GRAL PAZ  Y AV DE LOS CORRALES      

Hoja:  25%|██▌       | 1/4 [00:08<00:24,  8.19s/it]


Archivo: lesiones - Hoja: HECHOS
Columnas:
Index(['id', 'n_victimas', 'aaaa', 'mm', 'dd', 'fecha', 'hora', 'franja_hora',
       'direccion_normalizada', 'comuna', 'tipo_calle', 'otra_direccion',
       'calle', 'altura', 'cruce', 'geocodificacion_CABA', 'longitud',
       'latutid', 'victima', 'acusado', 'participantes', 'moto', 'auto',
       'transporte_publico', 'camion', 'ciclista', 'gravedad'],
      dtype='object')

Archivo: lesiones - Hoja: DICCIONARIO_HECHOS
Columnas:
Index(['variables y definiciones', 'descripción', 'Unnamed: 2'], dtype='object')


Hoja: 100%|██████████| 4/4 [00:12<00:00,  3.09s/it]


Archivo: lesiones - Hoja: VICTIMAS
Columnas:
Index(['ID hecho', 'AAA', 'MM', 'DD', 'FECHA ', 'VEHICULO_VICTIMA', 'SEXO',
       'EDAD_VICTIMA', 'GRAVEDAD'],
      dtype='object')

Archivo: lesiones - Hoja: DICCIONARIO_VICTIMAS
Columnas:
Index(['variables y definiciones', 'descripción', 'Unnamed: 2'], dtype='object')



Contenido de la hoja HECHOS en lesiones.xlsx:
                id  n_victimas  aaaa  mm  dd                fecha      hora  \
0  LC-2019-0000179           1  2019   1   1  2019-01-01 00:00:00  09:00:00   
1  LC-2019-0000053           1  2019   1   1  2019-01-01 00:00:00  01:55:00   
2  LC-2019-0000063           1  2019   1   1  2019-01-01 00:00:00  02:00:00   
3  LC-2019-0000079           1  2019   1   1  2019-01-01 00:00:00  02:30:00   
4  LC-2019-0000082           4  2019   1   1  2019-01-01 00:00:00  04:30:00   

  franja_hora direccion_normalizada comuna  ...     latutid   victima acusado  \
0           9                    SD     14  ...  -34.559658  CICLISTA      SD   
1           1                    SD      8  ...  -34.669125      AUTO      SD   
2           2                    SD      8  ...  -34.677556        SD      SD   
3           2                    SD      7  ...  -34.647349    PEATON      SD   
4           4                    SD      3  ...  -34.604579      AUTO    

Podemos observar que los archivos provistos por Ciudad de Buenos Aires llamados homicidios.xlsx y lesiones.xlsx contienen hojas o páginas, en el cual indicaré las hojas con las que trabajaré:

en el caso de homicidios: HECHOS y VICTIMAS

en el caso de lesiones: HECHOS y VICTIMAS

Por consiguiente preocederé a comparar el contenido de ambos archivos para realizar un merge.

También es importante considerar que hay una gran cantidad de datos nulos o en este caso 'sd' el cual analizaremos detenidamente.

<h4 style="color:red">
Trabajaré con homicidios & Lesiones

### Realizaré un merge en homicidios & lesiones.

LA finalidad es trabajar con toda la información unificada del excel en un solo df.

Lo que haré ahora es renombrar columnas, combinar los df y por último visualizar el merge.

In [4]:
def unir_hechos_victimas(hechos_df, victimas_df, sufijo_hechos, sufijo_victimas):
  """
  Une dos DataFrames (hechos y victimas) con sufijos.

  Args:
    hechos_df: DataFrame con datos de hechos.
    victimas_df: DataFrame con datos de victimas.
    sufijo_hechos: Sufijo para las columnas del DataFrame de hechos.
    sufijo_victimas: Sufijo para las columnas del DataFrame de victimas.

  Returns:
    DataFrame combinado con sufijos para las columnas.
  """

  # Renombro columnas de victimas_df
  columnas_victimas = victimas_df.columns
  nuevas_columnas_victimas = [f"{col}_{sufijo_victimas}" for col in columnas_victimas]
  victimas_df.columns = nuevas_columnas_victimas

  # Uno DataFrames
  df_unido = pd.concat([hechos_df, victimas_df], axis=1)

  return df_unido



In [5]:
# Uno homicidios_hechos y homicidios_victimas
homicidios_df = unir_hechos_victimas(homicidios_hechos, homicidios_victimas, "hechos", "victimas")

# Uno lesiones_hechos y lesiones_victimas
lesiones_df = unir_hechos_victimas(lesiones_hechos, lesiones_victimas, "hechos", "victimas")


In [6]:
# Agrego sufijos a las columnas
homicidios_df.columns = homicidios_df.columns.map(lambda x: x + "_homicidios")
lesiones_df.columns = lesiones_df.columns.map(lambda x: x + "_lesiones")

# Uno los DataFrames
siniestros_viales = pd.concat([homicidios_df, lesiones_df], axis=0)

# Verifico
siniestros_viales.info()


<class 'pandas.core.frame.DataFrame'>
Index: 28322 entries, 0 to 27604
Data columns (total 67 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   ID_homicidios                            696 non-null    object        
 1   N_VICTIMAS_homicidios                    696 non-null    float64       
 2   FECHA_homicidios                         696 non-null    datetime64[ns]
 3   AAAA_homicidios                          696 non-null    float64       
 4   MM_homicidios                            696 non-null    float64       
 5   DD_homicidios                            696 non-null    float64       
 6   HORA_homicidios                          696 non-null    object        
 7   HH_homicidios                            696 non-null    object        
 8   LUGAR_DEL_HECHO_homicidios               696 non-null    object        
 9   TIPO_DE_CALLE_homicidios                 696

In [7]:
# Visualizo
siniestros_viales

,ID_homicidios,N_VICTIMAS_homicidios,FECHA_homicidios,AAAA_homicidios,MM_homicidios,DD_homicidios,HORA_homicidios,HH_homicidios,LUGAR_DEL_HECHO_homicidios,TIPO_DE_CALLE_homicidios,...,gravedad_lesiones,ID hecho_victimas_lesiones,AAA_victimas_lesiones,MM_victimas_lesiones,DD_victimas_lesiones,FECHA _victimas_lesiones,VEHICULO_VICTIMA_victimas_lesiones,SEXO_victimas_lesiones,EDAD_VICTIMA_victimas_lesiones,GRAVEDAD_victimas_lesiones
0,2016-0001,1.0,2016-01-01,2016.0,1.0,1.0,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,2016-0002,1.0,2016-01-02,2016.0,1.0,2.0,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,2016-0003,1.0,2016-01-03,2016.0,1.0,3.0,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,2016-0004,1.0,2016-01-10,2016.0,1.0,10.0,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,2016-0005,1.0,2016-01-21,2016.0,1.0,21.0,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27600,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,LC-2021-0451911,2021.0,9.0,11.0,2021-09-11,TRANSPORTE PUBLICO,Varon,87,SD
27601,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,LC-2021-0530228,2021.0,10.0,25.0,2021-10-25,TRANSPORTE PUBLICO,Mujer,60,SD
27602,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,LC-2021-0530228,2021.0,10.0,25.0,2021-10-25,TRANSPORTE PUBLICO,Mujer,32,SD
27603,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,LC-2021-0201378,2021.0,5.0,2.0,2021-05-02,MOTO,Varon,32,SD


### Normalizaré la visualización a las reglas de capitalización de la Real Academia Española.

In [8]:
def normalizar_nombres_columnas(df):
  """
  Esta función normaliza los nombres de las columnas en un DataFrame de pandas.

  Parámetros:
  df (pandas.DataFrame): El DataFrame a normalizar.

  Retorna:
  df (pandas.DataFrame): El DataFrame con los nombres de las columnas normalizados.
  """

  # Convierto a minúsculas
  df.columns = df.columns.str.lower()

  # Elimino guiones bajos y reemplazar por espacios
  df.columns = df.columns.str.replace("_", " ")

  # Normalizo acentos
  df.columns = df.columns.str.normalize("NFKD")

  # Elimino caracteres que no sean letras o números
  df.columns = df.columns.str.replace("[^a-zA-Z0-9 ]", "")

  # Convierto la primera letra a mayúscula
  df.columns = df.columns.str.capitalize()

  return df

# Ejemplo de uso
siniestros_viales = normalizar_nombres_columnas(siniestros_viales.copy())

# Imprimo las columnas para verificar
print(siniestros_viales.columns)

Index(['Id homicidios', 'N victimas homicidios', 'Fecha homicidios',
       'Aaaa homicidios', 'Mm homicidios', 'Dd homicidios', 'Hora homicidios',
       'Hh homicidios', 'Lugar del hecho homicidios',
       'Tipo de calle homicidios', 'Calle homicidios', 'Altura homicidios',
       'Cruce homicidios', 'Dirección normalizada homicidios',
       'Comuna homicidios', 'Xy (caba) homicidios', 'Pos x homicidios',
       'Pos y homicidios', 'Participantes homicidios', 'Victima homicidios',
       'Acusado homicidios', 'Id hecho victimas homicidios',
       'Fecha victimas homicidios', 'Aaaa victimas homicidios',
       'Mm victimas homicidios', 'Dd victimas homicidios',
       'Rol victimas homicidios', 'Victima victimas homicidios',
       'Sexo victimas homicidios', 'Edad victimas homicidios',
       'Fecha fallecimiento victimas homicidios', 'Id lesiones',
       'N victimas lesiones', 'Aaaa lesiones', 'Mm lesiones', 'Dd lesiones',
       'Fecha lesiones', 'Hora lesiones', 'Franja hora 

In [9]:
# Elimino columnas para limpiar el df
columnas_a_eliminar = ['Dd homicidios', 'Aaaa victimas homicidios', 'Mm victimas homicidios',
                       'Dd victimas homicidios', 'Dd lesiones', 'Aaa victimas lesiones', 'Mm victimas lesiones',
                       'Dd victimas lesiones', 'Hh homicidios', 'Franja hora lesiones']

siniestros_viales = siniestros_viales.drop(columns=columnas_a_eliminar)


In [10]:
# Renombro 'Aaaa...' por 'Año.......'
siniestros_viales.rename(columns={'Aaaa homicidios': 'Año homicidios'}, inplace=True)

# Renombro 'Mn....' por 'Mes.......'
siniestros_viales.rename(columns={'Mm homicidios': 'Mes homicidios'}, inplace=True)

# Renombro 'Aaaa....' por 'Año.......'
siniestros_viales.rename(columns={'Aaaa lesiones': 'Año lesiones'}, inplace=True)

# Renombro 'Mm....' por 'Mes.......'
siniestros_viales.rename(columns={'Mm lesiones': 'Mes lesiones'}, inplace=True)

# Renombro 'Latutid....' por 'Latitud.......'
siniestros_viales.rename(columns={'Latutid lesiones': 'Latitud lesiones'}, inplace=True)

# Renombro 'Fecha....' por 'Fecha .......'
siniestros_viales.rename(columns={'Fecha  victimas lesiones': 'Fecha victimas lesiones'}, inplace=True)

# Renombro 'Vehiculo....' por 'Vehiculo .......'
siniestros_viales.rename(columns={'Vehiculo victima victimas lesiones': 'Vehiculo victimas lesiones'}, inplace=True)

# Renombro 'Edad....' por 'Edad .......'
siniestros_viales.rename(columns={'Edad victima victimas lesiones': 'Edad victimas lesiones'}, inplace=True)


### Verifico y realizo un conteno de los valores nulos en mis df

In [11]:
# Analizo porcentajes

total = siniestros_viales.shape[0]
nulos = siniestros_viales.isnull().sum()
porcentaje_nulos = (nulos / total) * 100

# Imprimo tipo de dato
tipos_datos = siniestros_viales.dtypes

print("Columna\t\tNulos\tPorcentaje\tTipo de dato")
for columna in siniestros_viales.columns:
    print(f"{columna}\t{nulos[columna]}\t{porcentaje_nulos[columna]}\t{tipos_datos[columna]}")

Columna		Nulos	Porcentaje	Tipo de dato
Id homicidios	27626	97.54254643033684	object
N victimas homicidios	27626	97.54254643033684	float64
Fecha homicidios	27626	97.54254643033684	datetime64[ns]
Año homicidios	27626	97.54254643033684	float64
Mes homicidios	27626	97.54254643033684	float64
Hora homicidios	27626	97.54254643033684	object
Lugar del hecho homicidios	27626	97.54254643033684	object
Tipo de calle homicidios	27626	97.54254643033684	object
Calle homicidios	27627	97.54607725443118	object
Altura homicidios	28193	99.54452369182968	float64
Cruce homicidios	27797	98.1463173504696	object
Dirección normalizada homicidios	27634	97.57079302309158	object
Comuna homicidios	27626	97.54254643033684	float64
Xy (caba) homicidios	27626	97.54254643033684	object
Pos x homicidios	27626	97.54254643033684	object
Pos y homicidios	27626	97.54254643033684	object
Participantes homicidios	27626	97.54254643033684	object
Victima homicidios	27626	97.54254643033684	object
Acusado homicidios	27626	97.542546430

#### Es relevante destacar que, en siniestros viales, la columna 'Altura' puede tener ausencia de datos debido a la naturaleza de los incidentes, concentrándose en las esquinas o intersecciones. 


In [12]:
# Obtengo información sobre los tipos de datos de las columnas
tipos_datos = siniestros_viales.dtypes

# Recorro las columnas y mostramos el nombre y el tipo de dato de cada una
for columna, tipo_dato in tipos_datos.items():
    print(f"Columna: {columna}")
    print(f"Tipo de dato: {tipo_dato}")
    print("\n")


Columna: Id homicidios
Tipo de dato: object


Columna: N victimas homicidios
Tipo de dato: float64


Columna: Fecha homicidios
Tipo de dato: datetime64[ns]


Columna: Año homicidios
Tipo de dato: float64


Columna: Mes homicidios
Tipo de dato: float64


Columna: Hora homicidios
Tipo de dato: object


Columna: Lugar del hecho homicidios
Tipo de dato: object


Columna: Tipo de calle homicidios
Tipo de dato: object


Columna: Calle homicidios
Tipo de dato: object


Columna: Altura homicidios
Tipo de dato: float64


Columna: Cruce homicidios
Tipo de dato: object


Columna: Dirección normalizada homicidios
Tipo de dato: object


Columna: Comuna homicidios
Tipo de dato: float64


Columna: Xy (caba) homicidios
Tipo de dato: object


Columna: Pos x homicidios
Tipo de dato: object


Columna: Pos y homicidios
Tipo de dato: object


Columna: Participantes homicidios
Tipo de dato: object


Columna: Victima homicidios
Tipo de dato: object


Columna: Acusado homicidios
Tipo de dato: object


Column

### Trabajaré con la columnas que contienen año, mes, fecha y edad.

In [13]:
def analizo_columna_individual(df, columna, num_filas=5):
    """
    Esta función analiza cada columna individualmente de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    columna (str): El nombre de la columna a analizar.
    num_filas (int): Número de filas a mostrar en la salida. Por defecto, se establece en 5.
    
    Retorna:
    None
    """
    # Verifico si hay valores nulos
    num_nulls = df[columna].isnull().sum()
    
    # Obtengo un resumen de la columna
    resumen = df[columna].describe(percentiles=[.25, .5, .75])

    # Imprimo el tipo de datos de la columna
    tipo_datos = df[columna].dtype

    # Obtengo los valores únicos de la columna
    valores_unicos = df[columna].unique()

    # Imprimo la cantidad de datos faltantes
    print(f"Número de datos faltantes en la columna '{columna}': {num_nulls}")

    # Si la columna es de tipo datetime, imprimo información adicional
    if tipo_datos == 'datetime64[ns]':
        print(f"Columna '{columna}':\nNúmero de valores nulos = {num_nulls}\nTipo de datos = {tipo_datos}\nFecha Mínima = {df[columna].min()}\nFecha Máxima = {df[columna].max()}\n")
    else:
        print(f"Columna '{columna}':\n{resumen.head(num_filas)}\nNúmero de valores nulos = {num_nulls}\nTipo de datos = {tipo_datos}\nValores únicos = {valores_unicos[:num_filas]}\n")

# Lista de columnas a analizar
columnas_a_analizar = ['Fecha homicidios', 'Año homicidios', 'Mes homicidios', 'Hora homicidios', 
                        'Fecha victimas homicidios', 'Edad victimas homicidios', 'Fecha fallecimiento victimas homicidios', 
                        'Año lesiones', 'Mes lesiones', 'Fecha lesiones', 'Hora lesiones', 
                        'Fecha victimas lesiones', 'Edad victimas lesiones']

# Uso la función para las columnas seleccionadas del DataFrame siniestros_viales
for columna in columnas_a_analizar:
    analizo_columna_individual(siniestros_viales, columna)


Número de datos faltantes en la columna 'Fecha homicidios': 27626
Columna 'Fecha homicidios':
Número de valores nulos = 27626
Tipo de datos = datetime64[ns]
Fecha Mínima = 2016-01-01 00:00:00
Fecha Máxima = 2021-12-30 00:00:00

Número de datos faltantes en la columna 'Año homicidios': 27626
Columna 'Año homicidios':
count     696.000000
mean     2018.188218
std         1.683754
min      2016.000000
25%      2017.000000
Name: Año homicidios, dtype: float64
Número de valores nulos = 27626
Tipo de datos = float64
Valores únicos = [2016. 2017. 2018. 2019. 2020.]

Número de datos faltantes en la columna 'Mes homicidios': 27626
Columna 'Mes homicidios':
count    696.000000
mean       6.692529
std        3.571309
min        1.000000
25%        4.000000
Name: Mes homicidios, dtype: float64
Número de valores nulos = 27626
Tipo de datos = float64
Valores únicos = [1. 2. 3. 4. 5.]

Número de datos faltantes en la columna 'Hora homicidios': 27626
Columna 'Hora homicidios':
count          696
uniqu

### Basándome en los datos procesaré los SD, NaN por Nat y creo la columna 'Día de la semana de fallecimiento'

In [14]:
# Listo de columnas a tratar
columnas_a_tratar = ['Fecha homicidios', 'Hora homicidios', 'Fecha victimas homicidios', 
                     'Fecha fallecimiento victimas homicidios', 'Fecha lesiones', 
                     'Hora lesiones', 'Fecha victimas lesiones']

# Tratamiento de los valores en las columnas seleccionadas
for columna in columnas_a_tratar:
    # Cambiar 'NaN', 'SD', 'sd' a 'NaT'
    siniestros_viales[columna] = pd.to_datetime(siniestros_viales[columna], errors='coerce')


In [15]:
# Listo de columnas a tratar
columnas_a_tratar = ['Fecha homicidios', 'Fecha fallecimiento victimas homicidios']

# Tratamiento de los valores en las columnas seleccionadas
for columna in columnas_a_tratar:
    # Cambiar 'NaN', 'SD', 'sd' a 'NaT'
    siniestros_viales[columna] = pd.to_datetime(siniestros_viales[columna], errors='coerce')

# Mapeo de nombres de días en inglés a español
dias_en_ingles = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dias_en_espanol = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

# Creo la columna 'Día de fecha de fallecimiento' y mapeo los nombres al español
siniestros_viales['Día de fecha de fallecimiento'] = siniestros_viales['Fecha fallecimiento victimas homicidios'].dt.day_name()
siniestros_viales['Día de fecha de fallecimiento'] = siniestros_viales['Día de fecha de fallecimiento'].map(dict(zip(dias_en_ingles, dias_en_espanol)))

# Visualizo solo la columna 'Día de fecha de fallecimiento'
print(siniestros_viales['Día de fecha de fallecimiento'])



0        Viernes
1         Sábado
2        Domingo
3            NaN
4          Lunes
          ...   
27600        NaN
27601        NaN
27602        NaN
27603        NaN
27604        NaN
Name: Día de fecha de fallecimiento, Length: 28322, dtype: object


In [16]:
# Trato los datos vacíos o NaN a 'Sin dato'
siniestros_viales['Día de fecha de fallecimiento'].fillna('Sin dato', inplace=True)

# Creo la columna 'Día de fallecimiento'
siniestros_viales['Día de fallecimiento'] = siniestros_viales['Fecha fallecimiento victimas homicidios'].dt.day_name(locale='es')

# Imprimo las primeras filas para verificar
print(siniestros_viales[['Fecha fallecimiento victimas homicidios', 'Día de fallecimiento']].head())


  Fecha fallecimiento victimas homicidios Día de fallecimiento
0                              2016-01-01              Viernes
1                              2016-01-02               Sábado
2                              2016-01-03              Domingo
3                                     NaT                  NaN
4                              2016-02-01                Lunes


In [17]:
# Trato los datos vacíos o NaN a 'Sin dato'
siniestros_viales['Día de fallecimiento'].fillna('Sin dato', inplace=True)

### Determino el periodo del df

In [18]:
def rango_fechas(df, columna):
    """
    Esta función obtiene el rango de fechas (fecha mínima y máxima) de una columna en un DataFrame de pandas,
    excluyendo los valores nulos.

    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    columna (str): El nombre de la columna a analizar.

    Retorna:
    La fecha mínima y máxima en la columna especificada, excluyendo los valores nulos.
    """
    # Filtrar las filas que tienen valores de fecha no nulos en la columna
    df_filtered = df[df[columna].notnull() & df[columna].apply(lambda x: isinstance(x, pd.Timestamp))]

    if not df_filtered.empty:
        fecha_min = df_filtered[columna].min()
        fecha_max = df_filtered[columna].max()
        return fecha_min, fecha_max
    else:
        return None, None

# Lista de columnas de fecha a analizar
columnas_fecha_a_analizar = ['Fecha homicidios', 'Fecha lesiones']

# Uso la función para las columnas seleccionadas del DataFrame siniestros_viales
for columna in columnas_fecha_a_analizar:
    fecha_min, fecha_max = rango_fechas(siniestros_viales, columna)
    print(f"Columna '{columna}':")
    if fecha_min is not None and fecha_max is not None:
        print(f"Fecha mínima: {fecha_min}")
        print(f"Fecha máxima: {fecha_max}\n")
    else:
        print("No hay fechas en esta columna.\n")


Columna 'Fecha homicidios':
Fecha mínima: 2016-01-01 00:00:00
Fecha máxima: 2021-12-30 00:00:00

Columna 'Fecha lesiones':
Fecha mínima: 2019-01-01 00:00:00
Fecha máxima: 2021-12-31 00:00:00



Estamos viendo que tanto homicidios como lesiones el rango de en homicidios comienza en el año 2016 y en lesiones en el año 2019, solo cambia en la fecha máxima por un día.

La finalidad de determinar el periodo, es que tendré que importar un calendario para trabajar el EDA y en power BI

### Trabajaré con columnas relacionadas a domicilio.

In [19]:
def analizo_columna_individual(df, columna):
    """
    Esta función analiza cada columna individualmente de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    columna (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    # Verifico si hay valores nulos
    num_nulls = df[columna].isnull().sum()
    
    # Obtengo un resumen de la columna
    resumen = df[columna].describe()

    # Imprimo el tipo de datos de la columna
    tipo_datos = df[columna].dtype

    # Obtengo los valores únicos de la columna
    valores_unicos = df[columna].unique()

    print(f"Columna '{columna}':\n{resumen}\nNúmero de valores nulos = {num_nulls}\nTipo de datos = {tipo_datos}\nValores únicos = {valores_unicos}\n")

# Lista de columnas a analizar
columnas_a_analizar = ['Lugar del hecho homicidios', 'Tipo de calle homicidios', 'Calle homicidios', 'Altura homicidios', 
'Cruce homicidios', 'Dirección normalizada homicidios', 'Direccion normalizada lesiones', 'Tipo calle lesiones', 
'Otra direccion lesiones', 'Calle lesiones', 'Altura lesiones', 'Cruce lesiones']

# Uso la función para las columnas seleccionadas del DataFrame siniestros_viales
for columna in columnas_a_analizar:
    analizo_columna_individual(siniestros_viales, columna)


Columna 'Lugar del hecho homicidios':
count                                696
unique                               683
top       AV 27 DE FEBRERO Y AV ESCALADA
freq                                   4
Name: Lugar del hecho homicidios, dtype: object
Número de valores nulos = 27626
Tipo de datos = object
Valores únicos = ['AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ'
 'AV GRAL PAZ  Y AV DE LOS CORRALES' 'AV ENTRE RIOS 2034'
 'AV LARRAZABAL Y GRAL VILLEGAS CONRADO'
 'AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA '
 'AV 27 DE FEBRERO Y AV ESCALADA' 'NOGOYA Y JOAQUIN V. GONZALES'
 'AV GENERAL PAZ Y AV DE LOS CORRALES'
 'AV BELGRANO Y BERNARDO DE IRIGOYEN' 'AV ENTRE RIOS 1366'
 'AV SCALABRINI ORTIZ Y VERA'
 'AV REGIMIENTO DE PATRICIOS Y QUINQUELA MARTIN BENITO'
 'MARIANO ACOSTA Y ANA MARIA JANER'
 'LISANDRO DE LA TORRE Y SEVERO GARCIA GRANDE DE ZEQUEIRA ' 'SUIPACHA 156'
 'AV DIRECTORIO Y CURAPALIGUE' 'AV 9 DE JULIO Y ADOLFO ALSINA'
 'AV GRAL PAZ Y AV 27 DE FEBRERO' 'HOLMBERG Y VEDIA'
 'CURAPALIGUE

Observamos que en dónde ocurren una mayor parte de siniestralidad es en un cruce llamado: AV 27 DE FEBRERO Y AV ESCALADA

In [20]:
# Considerando dicha salida procederé a tratar los nulos a renombrarlos a 'Sin Dato' 
columnas_a_tratar = ['Lugar del hecho homicidios', 'Tipo de calle homicidios', 'Calle homicidios', 'Altura homicidios', 
'Cruce homicidios', 'Dirección normalizada homicidios', 'Direccion normalizada lesiones',
'Tipo calle lesiones', 'Otra direccion lesiones', 'Calle lesiones', 'Altura lesiones', 'Cruce lesiones']

# Tratamiento de los valores nulos en las columnas seleccionadas
for columna in columnas_a_tratar:
    siniestros_viales[columna].fillna(0, inplace=True)


### Trabajaré con comunas.

In [21]:
# Función para obtener los primeros 20 valores numéricos no nulos en una columna
def obtener_valores_numericos_no_nulos(columna):
    """
    Esta función toma una columna y devuelve una lista con los primeros 20 valores numéricos no nulos.
    """
    valores_numericos = []
    for valor in columna:
        if pd.notna(valor):
            try:
                # Intentar convertir el valor a número
                valor_numerico = pd.to_numeric(valor)
                valores_numericos.append(valor_numerico)
                
                # Si se han encontrado 20 valores numéricos, salir del bucle
                if len(valores_numericos) == 20:
                    break
            except ValueError:
                # Ignorar valores no numéricos
                pass
    return valores_numericos

# Obtener los primeros 20 valores numéricos no nulos en 'Comuna homicidios'
valores_homicidios = obtener_valores_numericos_no_nulos(siniestros_viales['Comuna homicidios'])
print(f'Valores numéricos no nulos en Comuna homicidios: {valores_homicidios}')

# Obtener los primeros 20 valores numéricos no nulos en 'Comuna lesiones'
valores_lesiones = obtener_valores_numericos_no_nulos(siniestros_viales['Comuna lesiones'])
print(f'Valores numéricos no nulos en Comuna lesiones: {valores_lesiones}')


Valores numéricos no nulos en Comuna homicidios: [8.0, 9.0, 1.0, 8.0, 1.0, 8.0, 11.0, 9.0, 1.0, 1.0, 15.0, 4.0, 8.0, 9.0, 1.0, 7.0, 1.0, 8.0, 12.0, 7.0]
Valores numéricos no nulos en Comuna lesiones: [14, 8, 8, 7, 3, 15, 11, 9, 1, 4, 7, 11, 4, 1, 9, 12, 9, 2, 1, 5]


In [22]:
# Mostrar la cantidad de valores únicos en las columnas 'Comuna homicidios' y 'Comuna lesiones'
print("Valores únicos en Comuna homicidios:", siniestros_viales['Comuna homicidios'].unique())
print("Valores únicos en Comuna lesiones:", siniestros_viales['Comuna lesiones'].unique())



Valores únicos en Comuna homicidios: [ 8.  9.  1. 11. 15.  4.  7. 12.  3. 13. 14. 10.  6.  2.  5.  0. nan]
Valores únicos en Comuna lesiones: [nan 14 8 7 3 15 11 9 1 4 12 2 5 6 13 10 'No Especificada' 'SD']


In [23]:
# Reemplazar valores específicos
siniestros_viales['Comuna lesiones'].replace('No Especificada', 'Sin dato', inplace=True)
siniestros_viales['Comuna lesiones'].replace('SD', 'Sin dato', inplace=True)

# Llenar valores NaN con "Sin dato"
siniestros_viales['Comuna lesiones'].fillna('Sin dato', inplace=True)

# Convertir la columna a tipo categórico
siniestros_viales['Comuna lesiones'] = siniestros_viales['Comuna lesiones'].astype('category')


### Ya visualicé las columnas referidas a 'Comuna.....', ahora veremos el dataset de comunas, las ciudades que las integran, el cual es infomración oficial del Senso Nacional de Estadísticas y Censo de Argentina.

In [24]:
# Ruta del archivo que leere para ver datos estadísticos de la población y ciudades de cada comuna
archivo = 'C:/Users/Esteban García/OneDrive/Escritorio/LABs/PI02/Datasets Originales/c2022_caba_est_c1_1 normalizado.xlsx'

# Leo el archivo
xls = pd.ExcelFile(archivo)

# Veo las hojas disponibles en el archivo
print(xls.sheet_names)


['Carátula', 'Índice', 'Cuadro 1.1']


In [25]:
# Leo la hoja 'Cuadro 1.1' en un DataFrame
comunas = pd.read_excel(archivo, sheet_name='Cuadro 1.1')

# Visualizo
print(comunas.head())


   ID Comuna  Código Comuna  \
0       2007              1   
1       2014              2   
2       2021              3   
3       2028              4   
4       2035              5   

                     Ciudades que integran la comuna  \
0  Retiro, San Nicolás, Puerto Madero, San Telmo,...   
1                                           Recoleta   
2                           Balvanera, San Cristóbal   
3  La Boca, Barracas, Parque Patricios, Nueva Pom...   
4                                     Almagro, Boedo   

          Población Comuna año 2010   Total Población Comuna año 2010   \
0  Total población Comuna 1 año 2010                            205886   
1  Total población Comuna 2 año 2010                            157932   
2  Total población Comuna 3 año 2010                            187537   
3  Total población Comuna 4 año 2010                            218245   
4  Total población Comuna 5 año 2010                            179005   

           Población Comuna año

Aquí lo que hice, fue importar un excel de las comunas, las ciudades que las integran y la población, información que condiero importantísima para los próximos analisis que realizaré.

### Fuente: INDEC, Censo Nacional de Población, Hogares y Viviendas 2022. Resultados definitivos.

Notas: de acuerdo con la Ley Orgánica de Comunas Nº 1.777 sancionada en el 2005, la Ciudad Autónoma de Buenos Aires está organizada en 15 unidades descentralizadas de gestión político-administrativa que, en algunos casos, abarcan más de un barrio. Cada comuna corresponde a: Comuna 1: Retiro, San Nicolás, Puerto Madero, San Telmo, Montserrat y Constitución. Comuna 2: Recoleta. Comuna 3: Balvanera y San Cristóbal. Comuna 4: La Boca, Barracas, Parque Patricios y Nueva Pompeya. Comuna 5: Almagro y Boedo. Comuna 6: Caballito. Comuna 7: Flores y Parque Chacabuco. Comuna 8: Villa Soldati, Villa Riachuelo y Villa Lugano. Comuna 9: Liniers, Mataderos y Parque Avellaneda. Comuna 10: Villa Real, Monte Castro, Versalles, Floresta, Vélez Sarsfield y Villa Luro. Comuna 11: Villa General Mitre, Villa Devoto, Villa del Parque y Villa Santa Rita. Comuna 12: Coghlan, Saavedra, Villa Urquiza y Villa Pueyrredón. Comuna 13: Núñez, Belgrano y Colegiales. Comuna 14: Palermo. Comuna 15: Chacarita, Villa Crespo, La Paternal, Villa Ortúzar, Agronomía y Parque Chas. 								


### Analizo el contenido de las columnas.

In [26]:
print("Valores únicos en 'Comuna homicidios':", siniestros_viales['Comuna homicidios'].unique())
print("Valores únicos en 'Comuna lesiones':", siniestros_viales['Comuna lesiones'].unique())
print("Valores únicos en 'Código Comuna':", comunas['Código Comuna'].unique())
print("Valores_únicos en 'Ciudades que integran la comuna':", comunas['Ciudades que integran la comuna'].unique())


Valores únicos en 'Comuna homicidios': [ 8.  9.  1. 11. 15.  4.  7. 12.  3. 13. 14. 10.  6.  2.  5.  0. nan]
Valores únicos en 'Comuna lesiones': ['Sin dato', 14, 8, 7, 3, ..., 2, 5, 6, 13, 10]
Length: 16
Categories (16, object): [1, 2, 3, 4, ..., 13, 14, 15, 'Sin dato']
Valores únicos en 'Código Comuna': [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
Valores_únicos en 'Ciudades que integran la comuna': ['Retiro, San Nicolás, Puerto Madero, San Telmo, Montserrat, Constitución.'
 'Recoleta' 'Balvanera, San Cristóbal'
 'La Boca, Barracas, Parque Patricios, Nueva Pompeya' 'Almagro, Boedo'
 'Caballito' 'Flores, Parque Chacabuco'
 'Villa Soldati, Villa Riachuelo, Villa Lugano'
 'Liniers, Mataderos, Parque Avellaneda'
 'Villa Real, Monte Castro, Versalles, Floresta, Vélez Sarsfield, Villa Luro'
 'Villa General Mitre, Villa Devoto, Villa del Parque, Villa Santa Rita'
 'Coghlan, Saavedra, Villa Urquiza, Villa Pueyrredón'
 'Núñez, Belgrano, Colegiales' 'Palermo'
 'Chacarita, Villa Crespo, La Pa

### Trabajaré con las columnas Xy, Pos, Latitud, Longitud y Geocodificación.

In [27]:
def analizo_columna_individual(df, columna):
    """
    Esta función analiza cada columna individualmente de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    columna (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    # Verifico si la columna está presente en el DataFrame
    if columna not in df.columns:
        print(f"La columna '{columna}' no está presente en el DataFrame.")
        return

    # Verifico si hay valores nulos
    num_nulls = df[columna].isnull().sum()
    
    # Obtengo un resumen de la columna
    resumen = df[columna].describe()

    # Imprimo el tipo de datos de la columna
    tipo_datos = df[columna].dtype

    # Obtengo los valores únicos de la columna
    valores_unicos = df[columna].unique()

    # Agrego información adicional específica para columnas de coordenadas
    if 'Latitud' in columna or 'Longitud' in columna or 'Pos' in columna or 'Geocodificacion' in columna:
        # Verifico si hay valores únicos igual a 0 (podría indicar problemas)
        num_zeros = (df[columna] == 0).sum()
        info_adicional = f"Número de valores igual a 0 = {num_zeros}"

    else:
        info_adicional = ""

    # Imprimo la información
    print(f"Columna '{columna}':\n{resumen}\nNúmero de valores nulos = {num_nulls}\nTipo de datos = {tipo_datos}\nValores únicos = {valores_unicos}\n{info_adicional}\n")

# Uso la función para la columna 'Xy (caba) homicidios' del DataFrame siniestros_viales
analizo_columna_individual(siniestros_viales, 'Xy (caba) homicidios')

# Uso la función para la columna 'Pos x homicidios' del DataFrame siniestros_viales
analizo_columna_individual(siniestros_viales, 'Pos x homicidios')

# Uso la función para la columna 'Pos y homicidios' del DataFrame siniestros_viales
analizo_columna_individual(siniestros_viales, 'Pos y homicidios')

# Uso la función para la columna 'Geocodificacion caba lesiones' del DataFrame siniestros_viales
analizo_columna_individual(siniestros_viales, 'Geocodificacion caba lesiones')

# Uso la función para la columna 'Latitud lesiones' del DataFrame siniestros_viales
analizo_columna_individual(siniestros_viales, 'Latitud lesiones')

# Uso la función para la columna 'Longitud lesiones' del DataFrame siniestros_viales
analizo_columna_individual(siniestros_viales, 'Longitud lesiones')  

# Imprimo el DataFrame después de realizar los cambios
print(siniestros_viales.columns)


Columna 'Xy (caba) homicidios':
count             696
unique            606
top       Point (. .)
freq               14
Name: Xy (caba) homicidios, dtype: object
Número de valores nulos = 27626
Tipo de datos = object
Valores únicos = ['Point (98896.78238426 93532.43437792)'
 'Point (95832.05571093 95505.41641999)'
 'Point (106684.29090040 99706.57687843)'
 'Point (99840.65224780 94269.16534422)'
 'Point (106980.32827929 100752.16915795)'
 'Point (101721.59002217 93844.25656649)'
 'Point (96545.87592078 102330.67262199)'
 'Point (107595.35084333 101797.50052813)'
 'Point (106616.41069662 100496.44662323)'
 'Point (102357.43746828 103343.52002839)'
 'Point (108661.59115206 98481.75445555)'
 'Point (101228.61094416 96623.72136186)'
 'Point (95727.37083199 97179.35560349)'
 'Point (107684.31807562 102519.17333314)'
 'Point (101074.94198885 99848.19369046)'
 'Point (107507.39502374 102046.08333077)'
 'Point (100077.95205948 91571.23307293)'
 'Point (97350.75566590 109083.19420265)'
 'Point 

### Compararé columnas.

In [28]:
# Selecciono las columnas de geolocalización en homicidios
columnas_geolocalizacion_homicidios = ['Xy (caba) homicidios', 'Pos x homicidios', 'Pos y homicidios']

# Selecciono las columnas de geolocalización en lesiones
columnas_geolocalizacion_lesiones = ['Geocodificacion caba lesiones', 'Longitud lesiones', 'Latitud lesiones']

# Obtengo los valores únicos en las columnas de geolocalización en homicidios
valores_unicos_homicidios = siniestros_viales[columnas_geolocalizacion_homicidios].stack().unique()

# Obtengo los valores únicos en las columnas de geolocalización en lesiones
valores_unicos_lesiones = siniestros_viales[columnas_geolocalizacion_lesiones].stack().unique()

# Comparo los valores únicos en ambas columnas
diferencias = set(valores_unicos_homicidios) - set(valores_unicos_lesiones)

# Imprimo los primeros 10 resultados
print("Valores únicos en columnas de geolocalización en homicidios (primeros 10):")
print(valores_unicos_homicidios[:10])

print("\nValores únicos en columnas de geolocalización en lesiones (primeros 10):")
print(valores_unicos_lesiones[:10])

print("\nDiferencias en valores únicos entre homicidios y lesiones (primeros 10):")
print(list(diferencias)[:10])



Valores únicos en columnas de geolocalización en homicidios (primeros 10):
['Point (98896.78238426 93532.43437792)' '-58.47533969' '-34.68757022'
 'Point (95832.05571093 95505.41641999)' '-58.50877521' '-34.66977709'
 'Point (106684.29090040 99706.57687843)' '-58.39040293' '-34.63189362'
 'Point (99840.65224780 94269.16534422)']

Valores únicos en columnas de geolocalización en lesiones (primeros 10):
['Point (104991.46333918849995825 107720.96647481426771265)' '-58.408911'
 '-34.559658' 'Point (101813.84712503915943671 95578.55507230378862005)'
 '-58.44351' '-34.669125'
 'Point (99538.58660300650808495 94643.41871799012005795)' '-58.468335'
 '-34.677556' 'Point (102372.16830963784013875 97994.13750300348328892)']

Diferencias en valores únicos entre homicidios y lesiones (primeros 10):
['-58.36607137', '-58.43052224', 'Point (105219.14271583 101089.92439911)', '-58.44034689', '-58.49485242', 'Point (101025.65409402 109205.71453455)', '-58.37627355', '-58.52932872', '-34.63343907', 'Po

### Elimino y normalizo columnas referidas a geolocalización limpiando el df

In [29]:
# Eliminamos la columna 'Geocodificacion caba lesiones'
siniestros_viales = siniestros_viales.drop('Geocodificacion caba lesiones', axis=1)

# Normalizamos las coordenadas en 'Xy (caba) homicidios'
siniestros_viales['Xy (caba) homicidios'] = siniestros_viales['Xy (caba) homicidios'].astype(str)
siniestros_viales['Xy (caba) homicidios'] = siniestros_viales['Xy (caba) homicidios'].apply(lambda x: ', '.join(re.findall(r'-?\d+\.\d+', x)) if pd.notnull(x) else None)

# Renombramos la columna 'Xy (caba) homicidios'
siniestros_viales = siniestros_viales.rename(columns={'Xy (caba) homicidios': 'Coordenadas (CABA)'})

# Mostramos las primeras filas del DataFrame resultante
print(siniestros_viales[['Coordenadas (CABA)']])



                     Coordenadas (CABA)
0        98896.78238426, 93532.43437792
1        95832.05571093, 95505.41641999
2       106684.29090040, 99706.57687843
3        99840.65224780, 94269.16534422
4      106980.32827929, 100752.16915795
...                                 ...
27600                                  
27601                                  
27602                                  
27603                                  
27604                                  

[28322 rows x 1 columns]


### Relleno valores faltantes, tratando de eliminar la menor cantidad de filas.

In [30]:
# Relleno los valores nulos o vacíos en la columna 'Coordenadas (CABA)' con "No disponible"
siniestros_viales['Coordenadas (CABA)'] = siniestros_viales['Coordenadas (CABA)'].fillna("No disponible")

# Visualizo las primeras filas del DataFrame resultante
print(siniestros_viales[['Coordenadas (CABA)']])


                     Coordenadas (CABA)
0        98896.78238426, 93532.43437792
1        95832.05571093, 95505.41641999
2       106684.29090040, 99706.57687843
3        99840.65224780, 94269.16534422
4      106980.32827929, 100752.16915795
...                                 ...
27600                                  
27601                                  
27602                                  
27603                                  
27604                                  

[28322 rows x 1 columns]


In [31]:
# Relleno los valores nulos o vacíos en 'Coordenadas (CABA)' con "No disponible"
siniestros_viales['Coordenadas (CABA)'] = siniestros_viales['Coordenadas (CABA)'].fillna("No disponible")


In [32]:
# Modifico el nombre de la columna 'Coordenadas (CABA)'
siniestros_viales = siniestros_viales.rename(columns={'Coordenadas (CABA)': 'Coordenadas (CABA) homicidios'})

# Verifico idoneidad de 'Longitud lesiones' y 'Latitud lesiones'

# Elimino columnas 'Pos x homicidios' y 'Pos y homicidios'
siniestros_viales = siniestros_viales.drop(['Pos x homicidios', 'Pos y homicidios'], axis=1)


Podemos afirmar que las coordenas están normalizadas para el proceso de mapeo.

### Trabajaré con 'Edad.....'

In [33]:
def analizo_columna_individual(df, columna):
    """
    Esta función analiza cada columna individualmente de un DataFrame de pandas.
    
    Parámetros:
    df (pandas.DataFrame): El DataFrame a analizar.
    columna (str): El nombre de la columna a analizar.
    
    Retorna:
    None
    """
    # Verifico si hay valores nulos
    num_nulls = df[columna].isnull().sum()
    
    # Obtengo un resumen de la columna
    resumen = df[columna].describe()

    # Imprimo el tipo de datos de la columna
    tipo_datos = df[columna].dtype

    # Obtengo los valores únicos de la columna
    valores_unicos = df[columna].unique()

    # Imprimo el tipo de datos de 'SD' y nulos
    sd_values = df[columna][df[columna] == 'SD']
    nan_values = df[columna][df[columna].isnull()]
    tipo_datos_sd = type(sd_values.iloc[0]) if not sd_values.empty else None
    tipo_datos_nan = type(nan_values.iloc[0]) if not nan_values.empty else None

    print(f"Columna '{columna}':\n{resumen}\nNúmero de valores nulos = {num_nulls}\nTipo de datos = {tipo_datos}\nTipo de datos de 'SD' = {tipo_datos_sd}\nTipo de datos de nulos = {tipo_datos_nan}\nValores únicos = {valores_unicos}\n")

# Uso la función para la columna 'Edad victimas homicidios' del DataFrame siniestros_viales
analizo_columna_individual(siniestros_viales, 'Edad victimas homicidios')

# Uso la función para la columna 'Edad victimas lesiones' del DataFrame siniestros_viales
analizo_columna_individual(siniestros_viales, 'Edad victimas lesiones')


Columna 'Edad victimas homicidios':
count     717
unique     86
top        SD
freq       53
Name: Edad victimas homicidios, dtype: object
Número de valores nulos = 27605
Tipo de datos = object
Tipo de datos de 'SD' = <class 'str'>
Tipo de datos de nulos = <class 'float'>
Valores únicos = [19 70 30 18 29 22 16 59 65 34 41 50 38 21 52 36 20 54 'SD' 56 24 78 79 26
 57 37 58 23 60 42 53 51 40 87 76 75 35 80 43 45 67 27 55 49 81 25 33 46
 83 39 28 7 48 4 82 32 17 47 61 10 95 73 84 66 85 1 15 13 77 44 31 62 74
 71 11 86 69 72 12 63 92 68 91 64 5 88 nan]

Columna 'Edad victimas lesiones':
count     27605
unique      191
top          SD
freq       3990
Name: Edad victimas lesiones, dtype: object
Número de valores nulos = 717
Tipo de datos = object
Tipo de datos de 'SD' = <class 'str'>
Tipo de datos de nulos = <class 'float'>
Valores únicos = [nan 57 'SD' 45 27 32 21 37 33 25 23 42 53 1 62 67 75 40 48 3 49 '25' 34
 52 44 28 59 50 73 '26' 51 16 43 38 64 36 84 22 26 '32' 20 5 39 56 69 55
 46 29 4

In [34]:
# Reemplazo 'SD' y nulos con -1 en las columnas de edades
columnas_edades = ['Edad victimas homicidios', 'Edad victimas lesiones']
siniestros_viales[columnas_edades] = siniestros_viales[columnas_edades].replace({'SD': -1, np.nan: -1})

# Convierto las columnas a tipo numérico
siniestros_viales[columnas_edades] = siniestros_viales[columnas_edades].apply(pd.to_numeric, errors='coerce')

# Imprimo las columnas 'Edad victimas homicidios' y 'Edad victimas lesiones'
print(siniestros_viales[['Edad victimas homicidios', 'Edad victimas lesiones']])


       Edad victimas homicidios  Edad victimas lesiones
0                          19.0                    -1.0
1                          70.0                    -1.0
2                          30.0                    -1.0
3                          18.0                    -1.0
4                          29.0                    -1.0
...                         ...                     ...
27600                      -1.0                    87.0
27601                      -1.0                    60.0
27602                      -1.0                    32.0
27603                      -1.0                    32.0
27604                      -1.0                    41.0

[28322 rows x 2 columns]


### Analizaré si hay columnas duplicadas.

In [35]:
columnas_duplicadas = siniestros_viales.columns[siniestros_viales.columns.duplicated()]
print(columnas_duplicadas)


Index([], dtype='object')


### Trabajaré con participantes, victimas, rol, sexo

In [36]:
def analizar_dataframe(df, columnas):
    for columna in columnas:
        print(f"Análisis de la columna: {columna}\n{'-'*50}")
        
        # Tipo de datos
        tipo_datos = df[columna].dtype
        print(f"Tipo de datos: {tipo_datos}")
        
        # Número de valores únicos
        num_uniq = df[columna].nunique()
        print(f"Número de valores únicos: {num_uniq}")
        
        # Número de valores nulos
        num_null = df[columna].isnull().sum()
        print(f"Número de valores nulos: {num_null}")
        
        # Resumen de los valores no nulos
        if num_uniq > 0:
            print("Resumen de los valores (excluyendo nulos):")
            print(df[columna].dropna().describe())
        
        # Verificar si hay valores 'SD'
        num_sd = (df[columna] == 'SD').sum()
        print(f"Número de valores 'SD': {num_sd}")
        
        print("\n")

# Especifica las columnas que quieres analizar
columnas = ['Participantes homicidios', 'Victima homicidios', 'Acusado homicidios',
            'Rol victimas homicidios', 'Victima homicidios',
            'Sexo victimas homicidios', 'Victima lesiones', 'Acusado lesiones', 
            'Participantes lesiones', 'Moto lesiones', 'Auto lesiones', 
            'Transporte publico lesiones', 'Camion lesiones', 'Ciclista lesiones', 
            'Gravedad lesiones', 'Vehiculo victimas lesiones', 
            'Sexo victimas lesiones', 'Gravedad victimas lesiones']

# Uso la función para analizar las columnas especificadas del DataFrame siniestros_viales
analizar_dataframe(siniestros_viales, columnas)
def analizar_dataframe(df, columnas):
    for columna in columnas:
        print(f"Análisis de la columna: {columna}\n{'-'*50}")
        
        # Tipo de datos
        tipo_datos = df[columna].dtype
        print(f"Tipo de datos: {tipo_datos}")
        
        # Número de valores únicos
        num_uniq = df[columna].nunique()
        print(f"Número de valores únicos: {num_uniq}")
        
        # Imprimir los primeros 3 valores únicos
        print(f"Primeros 3 valores únicos: {df[columna].unique()[:3]}")
        
        # Número de valores nulos
        num_null = df[columna].isnull().sum()
        print(f"Número de valores nulos: {num_null}")
        
        # Resumen de los primeros 3 valores no nulos
        if num_uniq > 0:
            print("Primeros 3 valores no nulos:")
            print(df[columna].dropna().head(3))
        
        # Verificar si hay valores 'SD'
        num_sd = (df[columna] == 'SD').sum()
        print(f"Número de valores 'SD': {num_sd}")
        
        print("\n")

# Especifica las columnas que quieres analizar
columnas = ['Participantes homicidios', 'Victima homicidios', 'Acusado homicidios',
            'Rol victimas homicidios', 'Victima victimas homicidios',
            'Sexo victimas homicidios', 'Victima lesiones', 'Acusado lesiones', 
            'Participantes lesiones', 'Moto lesiones', 'Auto lesiones', 
            'Transporte publico lesiones', 'Camion lesiones', 'Ciclista lesiones', 
            'Gravedad lesiones', 'Vehiculo victimas lesiones', 
            'Sexo victimas lesiones', 'Gravedad victimas lesiones']

# Uso la función para analizar las columnas especificadas del DataFrame siniestros_viales
analizar_dataframe(siniestros_viales, columnas)



Análisis de la columna: Participantes homicidios
--------------------------------------------------
Tipo de datos: object
Número de valores únicos: 41
Número de valores nulos: 27626
Resumen de los valores (excluyendo nulos):
count                  696
unique                  41
top       PEATON-PASAJEROS
freq                   105
Name: Participantes homicidios, dtype: object
Número de valores 'SD': 0


Análisis de la columna: Victima homicidios
--------------------------------------------------
Tipo de datos: object
Número de valores únicos: 10
Número de valores nulos: 27626
Resumen de los valores (excluyendo nulos):
count      696
unique      10
top       MOTO
freq       295
Name: Victima homicidios, dtype: object
Número de valores 'SD': 9


Análisis de la columna: Acusado homicidios
--------------------------------------------------
Tipo de datos: object
Número de valores únicos: 10
Número de valores nulos: 27626
Resumen de los valores (excluyendo nulos):
count      696
unique      

### Procederé a normalizar

In [37]:
columnas_a_normalizar = ['Sexo victimas homicidios', 'Gravedad lesiones']
columnas_con_datos_faltantes = ['Participantes homicidios', 'Victima homicidios', 'Acusado homicidios']

siniestros_viales[columnas_a_normalizar] = siniestros_viales[columnas_a_normalizar].replace('SD', 'Sin dato')
siniestros_viales[columnas_con_datos_faltantes] = siniestros_viales[columnas_con_datos_faltantes].fillna('Sin dato')


### Optimizo para tener una buena práctica en términos de eficiencia modificando a category.

In [38]:
# Listo de columnas categóricas
columnas_categoricas = ['Participantes homicidios', 'Victima homicidios',
                         'Acusado homicidios', 'Rol victimas homicidios',
                         'Sexo victimas homicidios', 'Victima lesiones',
                         'Acusado lesiones', 'Participantes lesiones',
                         'Moto lesiones', 'Auto lesiones', 'Transporte publico lesiones',
                         'Camion lesiones', 'Ciclista lesiones', 'Gravedad lesiones',
                         'Vehiculo victimas lesiones', 'Sexo victimas lesiones',
                         'Gravedad victimas lesiones']

# Convierto columnas categóricas a tipo 'category'
siniestros_viales[columnas_categoricas] = siniestros_viales[columnas_categoricas].astype('category')


In [39]:
# Visualizo el df después de los cambios
print(siniestros_viales[['Participantes homicidios', 'Victima homicidios',
                         'Acusado homicidios', 'Rol victimas homicidios',
                         'Sexo victimas homicidios', 'Victima lesiones',
                         'Acusado lesiones', 'Participantes lesiones',
                         'Moto lesiones', 'Auto lesiones', 'Transporte publico lesiones',
                         'Camion lesiones', 'Ciclista lesiones', 'Gravedad lesiones',
                         'Vehiculo victimas lesiones', 'Sexo victimas lesiones',
                         'Gravedad victimas lesiones']])


      Participantes homicidios Victima homicidios Acusado homicidios  \
0                    MOTO-AUTO               MOTO               AUTO   
1               AUTO-PASAJEROS               AUTO          PASAJEROS   
2                    MOTO-AUTO               MOTO               AUTO   
3                      MOTO-SD               MOTO                 SD   
4               MOTO-PASAJEROS               MOTO          PASAJEROS   
...                        ...                ...                ...   
27600                 Sin dato           Sin dato           Sin dato   
27601                 Sin dato           Sin dato           Sin dato   
27602                 Sin dato           Sin dato           Sin dato   
27603                 Sin dato           Sin dato           Sin dato   
27604                 Sin dato           Sin dato           Sin dato   

      Rol victimas homicidios Sexo victimas homicidios Victima lesiones  \
0                   CONDUCTOR                MASCULINO      

In [40]:
# Listo columnas con valores 'SD'
columnas_sd = ['Participantes homicidios', 'Victima homicidios',
                'Acusado homicidios', 'Rol victimas homicidios',
                'Sexo victimas homicidios', 'Victima lesiones',
                'Acusado lesiones', 'Participantes lesiones',
                'Moto lesiones', 'Auto lesiones', 'Transporte publico lesiones',
                'Camion lesiones', 'Ciclista lesiones', 'Gravedad lesiones',
                'Vehiculo victimas lesiones', 'Sexo victimas lesiones',
                'Gravedad victimas lesiones']

# Reemplazo 'SD' con 'Sin Dato' en las columnas especificadas
siniestros_viales[columnas_sd] = siniestros_viales[columnas_sd].replace('SD', 'Sin dato')

# Unificar representación de valores faltantes
siniestros_viales.replace('Sin dato', np.nan, inplace=True)


### Ya he trabajado en los df homicidios y lesiones, ahora crearé el csv

In [41]:
# Ruta donde se guardará el archivo CSV
ruta_guardado = r'C:\Users\Esteban García\OneDrive\Escritorio\LABs\PI02\Df procesados en ETL\siniestros_viales_procesado.csv'

# Guardo el df en un archivo CSV
siniestros_viales.to_csv(ruta_guardado, index=False)
